## Bibliotecas

In [1]:
import pandas as pd
import ast

## Importadores

In [2]:
# Lista de importadores de capacete de moto
importadorMot = pd.read_excel('https://raw.githubusercontent.com/FredericoCS1/Importacoes/main/Importadores/importadoresMoto.xlsx')
importadorMot.head()

,ImportadoresMoto
0,"BR MOTORSPORT COMERCIO, IMPORTACAO E EXPORTACA..."
1,EUROSTAR DO BRASIL S/A.
2,CAPITAL TRADE IMPORTACAO E EXPORTACAO LTDA
3,"BR MOTORSPORT COMERCIO, IMPORTACAO E EXPORTACA..."
4,SILVA MATTOS & CIA LTDA


In [3]:
# Transformar o DataFrame em lista
importadorMot = importadorMot['ImportadoresMoto'].to_list()

In [4]:
# Lista de importadores adicionais
importadorExt = pd.read_excel('https://raw.githubusercontent.com/FredericoCS1/Importacoes/main/Importadores/importadoresExtra.xlsx')
importadorExt.head()

,ImportadoresExtra
0,EMBRAER S.A.
1,AMAZON SERVICOS DE VAREJO DO BRASIL LTDA.
2,HONEYWELL INDUSTRIA DE TECNOLOGIA LTDA.
3,SERTRADING (BR) LTDA.
4,PIT LANE ACESSORIOS ESPORTIVOS EIRELI


In [5]:
# Transformar o DataFrame em lista
importadorExt = importadorExt['ImportadoresExtra'].to_list()

## Código

In [6]:
# Leitura da base, retornando os primeiros valores
df = pd.read_excel('Relatório de Importação.xlsx')
df.head()

,PAIS DE ORIGEM,País de aquisição,ANO/MÊS,QTD Estatística,Unidade de Medida Estatística,Peso líquido,Unidade de Medida Comercializada Estimada,Valor FOB Estimado Unitário,VALOR FOB ESTIMADO TOTAL,Valor CIF Unitário,...,NCM,MODAL,PROVÁVEL IMPORTADOR,PROVÁVEL EXPORTADOR,PROVÁVEL IMPORTADOR CNPJ,Notificado,CIDADE DO IMP.,UF IMP,Palavras chave de descrição do produto,Relevância por palavra-chave
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CHINA,CHINA,202409.0,30.0,NÚMERO (UNIDADE),2.0,NÚMERO (UNIDADE),2.00,60.0,2.03,...,65061000.0,MARITIMA,3 MASTER COMERCIO ATACADISTA LTDA.,YIWU MENGYING IMPORT E EXPORT CHINA,42.087.674/0001-15,NaN,SANTANA DO IPANEMA,AL,"[ 'SOLDA' , 'LENTES' , 'REGULAGEM' , '22A23F' ...","{0: {'keywords':['110X90MM','22A23F','ACEITA',..."
2,SUÍÇA,SUÍÇA,202312.0,72.0,NÚMERO (UNIDADE),32.4,NÚMERO (UNIDADE),30.84,2220.4,31.86,...,65061000.0,MARITIMA,3M DO BRASIL LTDA,3M EMEA PL ZOO IATD SWITZERLAND,45.985.371/0001-08,NaN,SUMARÉ,SP,"[ 'PROTECAO' , '52000182023' , 'POLIFTALAMIDA'...","{0: {'keywords':['52000182023','CAPACETE','HEL..."
3,SUÉCIA,SUÍÇA,202312.0,888.0,NÚMERO (UNIDADE),400.1,NÚMERO (UNIDADE),30.84,27385.5,31.47,...,65061000.0,MARITIMA,3M DO BRASIL LTDA,3M EMEA PL ZOO IATD SWITZERLAND,45.985.371/0001-08,NaN,SUMARÉ,SP,"[ 'PROTECAO' , '52000182023' , 'POLIFTALAMIDA'...","{0: {'keywords':['52000182023','CAPACETE','HEL..."
4,ESTADOS UNIDOS,ESTADOS UNIDOS,202312.0,120.0,NÚMERO (UNIDADE),68.6,NÚMERO (UNIDADE),118.33,14199.5,118.74,...,65061000.0,MARITIMA,3M DO BRASIL LTDA,3M UNITED STATES,45.985.371/0001-08,NaN,SUMARÉ,SP,"[ 'VISOR' , '70070121515' , 'CONJUNTO' , 'CONF...","{0: {'keywords':['3M','9B7BBF','70070121515','..."


In [7]:
# Retorna a quantidade de valores de uma coluna da base
df['Palavras chave de descrição do produto'].count()

1087

In [8]:
# Tirando a primeira linha da base, porque só contem valores nulos e sem importância
df.drop([0, 0], inplace=True)

In [9]:
# Criando uma cópia da base para estudo, assim não afeta a principal
dfCap = df.copy()

In [10]:
# Dentro da cópia chamada de 'dfCap', foi criada uma variável, 'filterCap', esta irá armazenar as modificações feitas na base cópia
# Esta parte faz uma busca dentro da coluna, 'Palavras chave de descrição do produto', para os valores dentro de 'contains', todos os valores da base são considerados string
filterCap = dfCap[
    dfCap['Palavras chave de descrição do produto']
    .astype(str)
    .str.contains(r'CAPACETE|HELMET', na=False)
# Esta segunda parte faz uma busca na mesma coluna, mas agora retirando os valores dentro de 'contains', valores para string
    & ~dfCap['Palavras chave de descrição do produto']
    .astype(str)
    .str.contains(r'PROTETOR|BOMBEIRO|INCENDIO|SKI|INDUSTRIAL|'
                  r'EQUITAÇÃO|MASCARA|EPI|AUTOGRAFADO', na=False)
]
# Para a criação de uma novo DF, tabela, será retirado os index, para que não afetem a leitura
filterCap.reset_index(drop=True, inplace=True)
# Retorna a quantidade de valores de uma coluna da base
filterCap['Palavras chave de descrição do produto'].count()

790

In [11]:
# Transformação do DF em excel
filterCap.to_excel('Relatório de Importação de Capacetes.xlsx', index=False)

In [12]:
# Leitura da nova base, retornando a quantidade de valores
df2 = pd.read_excel('Relatório de Importação de Capacetes.xlsx')
df2['Palavras chave de descrição do produto'].count()

790

In [13]:
# Cópia da base
dfAj = df2.copy()

In [14]:
# Armazena as modificações
# Retira os valores dentro de 'contains', string
filterAj = dfAj[
    ~dfAj['Palavras chave de descrição do produto']
    .astype(str)
    .str.contains(r'BICICLETA|AUTOMOBILISMO|MOUNTAIN|TREK|SCOTT', na=False)
]

# Retira index
filterAj.reset_index(drop=True, inplace=True)
# Quantidade de valores
filterAj['Palavras chave de descrição do produto'].count()

749

In [15]:
# DF para excel
filterAj.to_excel('Relatório de Importação de Capacetes Ajustado.xlsx', index=False)

In [16]:
# Nova base, retornando a quantidade de valores
df3 = pd.read_excel('Relatório de Importação de Capacetes Ajustado.xlsx')
df3['Palavras chave de descrição do produto'].count()

749

In [17]:
# Cópia da base
dfFim = df3.copy()

In [18]:
# A filtragem desta parte consiste na busca dos valores dentro da lista 'importadorMot'
# Aplica uma função lambda, para cada elemento 'x', da coluna 'PROVÁVEL IMPORTADOR'
# 'x' representa um valor único da coluna, passado na função lammbda
# any, é uma expressão para verificar se existe algum valor string dentro da lista 'importadorMot' que é subvalor, substring, de 'x'
# Verifica se existe algum elemento 'imp' dentro de 'importadorMot', que esta contido em 'x'
filterFim = dfFim[dfFim['PROVÁVEL IMPORTADOR']
    .astype(str)
    .apply(lambda x: any(imp in x for imp in importadorMot))
]
# Retira index
filterFim.reset_index(drop=True, inplace=True)
# Quantidade de valores
filterFim['Palavras chave de descrição do produto'].count()

191

In [19]:
# DF para excel
filterFim.to_excel('Relatório de Importação de Capacetes de Moto.xlsx', index=False)

In [20]:
# Leitura da base, retornando a quantidade de valores
df4 = pd.read_excel('Relatório de Importação de Capacetes de Moto.xlsx')
df4['Palavras chave de descrição do produto'].count()

191

In [21]:
# Cópia da base
dfRes = df4.copy()

In [22]:
# Lista vazia para os dados extraídos
extracted_data = []

# Função for que itera para cada linha do DataFrame
for index, row in dfRes.iterrows():
  # Tenta executar as funções
  try:
    # Transforma a string em um dicionário Python
    row_dict = ast.literal_eval(row['Relevância por palavra-chave'])
    # Função for que itera cada parâmetro do dicionário
    for key, inner_dict in row_dict.items():
      # Para cada parâmetro do dicionário, um dado é extraído e colocado na lista
        extracted_data.append({
        'row_index': index,
        'Keywords': inner_dict['keywords'],
        'Quantidade Estatística': inner_dict['relevancia_qtd'],
        'Valor FOB Unitário': inner_dict['relevancia_fob']
    })
  # Se tiver um erro, separa dos extraídos para que não pare a execução
  except (ValueError, KeyError, SyntaxError) as e:
    print(f"Erro ao processar a linha {index}: {e}")

# Transforma a lista dos extraídos em um DataFrame
dfRes_extracted = pd.DataFrame(extracted_data)

# Transforma os valores para numérico
dfRes_extracted['Quantidade Estatística'] = pd.to_numeric(dfRes_extracted['Quantidade Estatística'], errors='coerce')
dfRes_extracted['Valor FOB Unitário'] = pd.to_numeric(dfRes_extracted['Valor FOB Unitário'], errors='coerce')

# Une esse novo DF com o resto da base
dfRes_final = dfRes.merge(dfRes_extracted, left_index=True, right_on = 'row_index', how='left')
# Excluí colunas indesejadas
dfRes_final.drop(columns=['Relevância por palavra-chave', 'row_index'], inplace=True)

dfRes_final.head()

,PAIS DE ORIGEM,País de aquisição,ANO/MÊS,QTD Estatística,Unidade de Medida Estatística,Peso líquido,Unidade de Medida Comercializada Estimada,Valor FOB Estimado Unitário,VALOR FOB ESTIMADO TOTAL,Valor CIF Unitário,...,PROVÁVEL IMPORTADOR,PROVÁVEL EXPORTADOR,PROVÁVEL IMPORTADOR CNPJ,Notificado,CIDADE DO IMP.,UF IMP,Palavras chave de descrição do produto,Keywords,Quantidade Estatística,Valor FOB Unitário
0,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW DO BRASIL LTDA,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","[AMARELO, BMW, CAPACETE, CARBONO, CORES, FECHA...",115,31681
1,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW DO BRASIL LTDA,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","[BMW, CAPACETE, CARBONO, CASCO, 58312F, CORES,...",114,31451
2,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW DO BRASIL LTDA,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","[BMW, BRANCO, CAPACETE, CARBONO, CORES, FECHAD...",85,23162
3,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW DO BRASIL LTDA,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","[BMW, BRANCO, CAPACETE, CARBONO, CASCO, CORES,...",72,18173
4,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW DO BRASIL LTDA,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...",[OUTROS],100,30748


In [23]:
# Divide a coluna FOB pela Quantidade estatística
dfRes_final['QTD por FOB'] = dfRes_final['Valor FOB Unitário'] / dfRes_final['Quantidade Estatística']
dfRes_final.head()

,PAIS DE ORIGEM,País de aquisição,ANO/MÊS,QTD Estatística,Unidade de Medida Estatística,Peso líquido,Unidade de Medida Comercializada Estimada,Valor FOB Estimado Unitário,VALOR FOB ESTIMADO TOTAL,Valor CIF Unitário,...,PROVÁVEL EXPORTADOR,PROVÁVEL IMPORTADOR CNPJ,Notificado,CIDADE DO IMP.,UF IMP,Palavras chave de descrição do produto,Keywords,Quantidade Estatística,Valor FOB Unitário,QTD por FOB
0,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","[AMARELO, BMW, CAPACETE, CARBONO, CORES, FECHA...",115,31681,275.486957
1,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","[BMW, CAPACETE, CARBONO, CASCO, 58312F, CORES,...",114,31451,275.885965
2,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","[BMW, BRANCO, CAPACETE, CARBONO, CORES, FECHAD...",85,23162,272.494118
3,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","[BMW, BRANCO, CAPACETE, CARBONO, CASCO, CORES,...",72,18173,252.402778
4,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...",[OUTROS],100,30748,307.480000


In [24]:
# DF para Excel
dfRes_final.to_excel('Relatório de Importação de Capacetes de Moto Final.xlsx', index=False)

In [25]:
# Nova base
df5 = pd.read_excel('Relatório de Importação de Capacetes de Moto Final.xlsx')
df5.head()

,PAIS DE ORIGEM,País de aquisição,ANO/MÊS,QTD Estatística,Unidade de Medida Estatística,Peso líquido,Unidade de Medida Comercializada Estimada,Valor FOB Estimado Unitário,VALOR FOB ESTIMADO TOTAL,Valor CIF Unitário,...,PROVÁVEL EXPORTADOR,PROVÁVEL IMPORTADOR CNPJ,Notificado,CIDADE DO IMP.,UF IMP,Palavras chave de descrição do produto,Keywords,Quantidade Estatística,Valor FOB Unitário,QTD por FOB
0,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","['AMARELO', 'BMW', 'CAPACETE', 'CARBONO', 'COR...",115,31681,275.486957
1,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","['BMW', 'CAPACETE', 'CARBONO', 'CASCO', '58312...",114,31451,275.885965
2,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","['BMW', 'BRANCO', 'CAPACETE', 'CARBONO', 'CORE...",85,23162,272.494118
3,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...","['BMW', 'BRANCO', 'CAPACETE', 'CARBONO', 'CASC...",72,18173,252.402778
4,TAILÂNDIA,ALEMANHA,202405,485,NÚMERO (UNIDADE),825.7,NÚMERO (UNIDADE),278.79,135215.2,284.72,...,BMW PARTS GERMANY,00.882.430/0006-99,NaN,CAJAMAR,SP,"[ 'VERSOES' , 'FECHADO' , 'CORES' , 'POLICARBO...",['OUTROS'],100,30748,307.480000


In [26]:
# Cópia da base
dfExt = df5.copy()

In [27]:
# Lista contendo marcas de capacete de moto
brandsMot = [
    "AGV", "AIROH", "ALPINESTAR", "ARAI", "ASW", "ASTONE", "AXXIS",
    "BIEFFE", "BMW", "GP TECH", "HELT", "HJC", "KYT", "LS2", "LUCCA",
    "NEXX", "NOLAN", "NORISK", "NZI", "PEELS", 'RACE', "RACE TECH",
    "SCHUBERTH", "SHARK", "SHOEI", "SMK", "STG", "SUOMY", "TEXX",
    "X11", 'Reaper']

In [28]:
# Vai trazer todos os valores que não estão contidos dentro da lista 'brandsMot'
filterExt = dfExt[
    ~dfExt['Palavras chave de descrição do produto']
    .astype(str)
    .str.contains('|'.join(brandsMot))]
# Retira index
filterExt.reset_index(drop=True, inplace=True)
# Quantidade de valores
filterExt['Palavras chave de descrição do produto'].count()

105